In [1]:
import numpy as np                          # Basic Array and Numaric Operation
import os                                   # use to access the files 
import tarfile                              # use to extract dataset from zip files
import sys
import zipfile                              # to extract zip file


import torch                                # Provides basic tensor operation and nn operation
import torchvision                          # Provides facilities to access image dataset


import my_utils.loadDataset as dl           # create dataloader for selected dataset
import my_utils.loadModel as lm             # facilitate loading and manipulating models
import my_utils.trainModel as tm            # Facilitate training of the model
import my_utils.initialize_pruning as ip    # Initialize and provide basic parmeter require for pruning
import my_utils.facilitate_pruning as fp    # Compute Pruning Value and many things

In [ ]:
def freeze(model,model_name):
    count = 0
    if model_name == 'vgg16':
        for param in model.parameters():
            if count == 30:
              param.requires_grad=True
            else:
              param.requires_grad=False
        count = count+1

### Prepare dataloader and set image properties. Check for cuda device and load model

In [2]:
# set the locationn of the dataset and trai and test data folder name
dl.setFolderLocation(datasets       ='/home/pragnesh/Dataset/',
                     selectedDataset='IntelIC/',
                     train          ='train',
                     test           ='test')
# set the imge properties
dl.setImageSize(224)
dl.setBatchSize = 2
dataLoader = dl.dataLoader()

#load the saved model if have any
load_path = "/home/pragnesh/Model/VGG_IntelIC_v2"
device1 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
newModel = torch.load(load_path, map_location=torch.device('cpu'))

#if dont have any saved trained model download pretrained model for tranfer learning
# newmodel = lm.load_model(model_name='vgg16',number_of_class=6,pretrainval=False,
#                         freeze_feature=False,device_l=device1)

#print(newModel)

### Here we have 3 set of parameter.One for data loader, second for model and third are hyperparameter
### Train the model

In [3]:
logFile = '/home/pragnesh/Dataset/Intel_Image_Classifacation_v2/Logs/ConvModelv2.log'
#tm.device = torch.device('cpu')
tm.fit_one_cycle(#set locations of the dataset, train and test data
                 dataloaders=dataLoader,trainDir=dl.trainDir,testDir=dl.testDir,
                 # Selecat a variant of VGGNet
                 ModelName='vgg16',model=newModel,device_l=device1,
                 # Set all the Hyper-Parameter for training
                 epochs=1, max_lr=0.01, weight_decay=0, L1=0, grad_clip=.1, logFile=logFile)

#Save the  trained model 
SavePath = '/home/pragnesh/Model/vgg16-v2'
torch.save(newModel, SavePath)

Training Starts


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

### Pruning Initialization

In [ ]:
blockList   = ip.createBlockList(newModel)              #ip.getBlockList('vgg16')
featureList = ip.createFeatureList(newModel)
convIdx     = ip.findConvIndex(newModel)
module      = ip.getPruneModule(newModel)
prune_count = ip.getPruneCount(module=module,blocks=blockList,maxpr=.1)
print(f"Block List   = {blockList}\n"
      f"Feature List = {featureList}\n" 
      f"Conv Index   = {convIdx}\n"
      f"Prune Count  = {prune_count}"
      
     )
for i in range(len(module)):
    print(f"{i+1} :{module[i]}")

### Pruning Process Starts From Here
##### compute score for pruning criteria for candidate convolution pruning layer

In [ ]:
candidateConvLayer = []
st =0
en = 2
for i in range(st,en):
    candidateConvLayer.append(fp.compute_distance_score(module[i]._parameters['weight'],threshold=1))
print("\n\n\nHere is the :",len(candidateConvLayer))

#### *Sort the list of score in the ascending order*

In [ ]:
for i in range(len(candidateConvLayer)):
    fp.sort_kernel_by_distance(candidateConvLayer[i])

In [ ]:
print(f'c={len(candidateConvLayer)}, c[] ={len(candidateConvLayer[0])},' 
      f'c[][] = {len(candidateConvLayer[0][0])},c[][][] = {len(candidateConvLayer[0][0][0])}')

#### *Extract K Elements of ascending List of Score and they will be the minimum score*

In [ ]:
prune_count = ip.getPruneCount(module=module,blocks=blockList,maxpr=0.1,)
newList = []
for i in range(len(candidateConvLayer)):
    newList.append(fp.get_k_element(channel_list=candidateConvLayer[i],k=prune_count[st+i])  )
print(f'c={len(newList)}, c[] ={len(newList[0])},' 
      f'c[][] = {len(newList[0][0])},c[][][] = {len(newList[0][0][0])}')

### Now Execute the pruning

#### Define class to execute custom pruning

In [ ]:
import torch.nn.utils.prune as prune
class KernalPruningMethod(prune.BasePruningMethod):
    PRUNING_TYPE = 'unstructured'
    def compute_mask(self, t, default_mask):
        mask = default_mask.clone()
        #mask.view(-1)[::2] = 0
        size = t.shape
        print(size)
        for k1 in range(len(newList[layer_number-st])):
            for k2 in range(len(newList[layer_number-st][k1])):
                i= newList[layer_number-st][k1][k2][1]
                j= newList[layer_number-st][k1][k2][0]
                
                #print(f"i= {i} , j= {j}")
                
                mask[i][j] = 0
        return mask
def kernal_unstructured(module, name):
    KernalPruningMethod.apply(module, name)
    return module

### *Pass the cadidate layer of Conv Net for custom pruning*

In [ ]:
layer_number = 0
for i in range(st,en):
    layer_number = i
    kernal_unstructured(module[i], name = 'weight')
    layer_number += 1

##### commit the pruning parameter 

In [ ]:
for i in range(st,en):
    prune.remove(module[i], 'weight')

### Deep Copy Code

In [ ]:
def updateFeatureList(featureList,prune_count,start,end):
    j=0
    i=st
    while j < en:
        if featureList[i] == 'M':
            i+=1
            continue
        else:
            featureList[i] = featureList[i] - prune_count[j]
            j+=1
            i+=1
    return featureList
featureList= updateFeatureList(featureList=featureList, prune_count=prune_count,start=st,end=en)


In [ ]:
print(featureList)

In [ ]:
#featureList = ip.createFeatureList(newModel)

In [ ]:
tempModel = lm.create_vgg_from_feature_list(featureList)
print(tempModel)

In [ ]:

count=0
for i in range(len(newModel.features)):
    if str(newModel.features[i]).find('Conv') != -1:
        size = newModel.features[i]._parameters['weight'].shape
        
        for fin in range(size[1]):
            j=0
            for fout in range(size[0]):
                if torch.norm(newModel.features[i]._parameters['weight'][fout][fin]) != 0:
                    print(f'i ={i} , j={j}, fin = {fin}, fout= {fout}')
                    if j>=featureList[count]:
                        break
                    tempModel.features[i]._parameters['weight'][j][fin]=newModel.features[i]._parameters['weight'][fout][fin]
                    j = j+1
        count = count +1    
        #print(newModel.features[i]._parameters['weight'].shape)

In [ ]:
tempModel

In [ ]:
newModel

In [ ]:
lm.freeze(tempModel,'vgg16')